<a href="https://colab.research.google.com/github/La-matrice/brouhaha/blob/main/entrainement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entrainement du modèle 'brouhaha'**
**1) phrases.txt ==> mots.txt**

*   Élément de liste
*   Élément de liste


**2) mots.txt ==> silence.txt**

*   Élément de liste
*   Élément de liste



# **A - Identification des mots saisis par l'utilisateur**